In [ ]:
import os
import sys
sys.path.append(os.path.dirname(os.path.realpath('../ux')))

from ux.jupyter_imports import *
import copy

In [ ]:
original_pnl = pd.read_csv(os.path.join(os.sep, 'tmp','pfoptimizer','pnl.csv'), 
                           date_parser=parser.parse, 
                           parse_dates=['end_time'],
                           index_col=0)
original_trajectory = pd.read_csv(os.path.join(os.sep, 'tmp','pfoptimizer','trajectory.csv'))
original_history = pd.read_csv(os.path.join(os.sep, 'tmp','pfoptimizer','history.csv'), 
                               date_parser=parser.parse, 
                               index_col=0)
equity = 1e7

In [ ]:
end = datetime.now().replace(tzinfo=timezone.utc)
start = end - timedelta(days = 365)
pnl = original_pnl[(original_pnl['end_time']>start)&(original_pnl['end_time']<end)]

carry = pnl[pnl['bucket']=='carry(USD not annualized)'].drop(columns='bucket')
carry['amtUSD'] *= 1e4/equity # in bps

weights = pnl[pnl['bucket']=='weights'].drop(columns='bucket')
weights['amtUSD'] /= equity

IR01 = pnl[pnl['bucket']=='IR01(USD)'].drop(columns='bucket')
IR01['amtUSD'] *= 1e4/equity # in bps

tx_cost = pnl[pnl['bucket']=='tx_cost(USD not annualized, fwd)'].drop(columns='bucket')
tx_cost['amtUSD'] *= 1e4/equity # in bps

# Excess return in bps
## assumes spreads are done at mid every funding period
## Pls note there is no hyperparamters (so no overfit here)
### * carry = funding-borrow
### * IR01 = spot*premium chg
### * delta = 0 (only exists during exec)

In [ ]:
pnl_buckets = pnl[(pnl['name']=='total')&(pnl['bucket']!='weights')].pivot_table(index='end_time',columns='bucket',values='amtUSD').cumsum()
(pnl_buckets* 1e4/equity).iplot(title=f'pnl explain', secondary_y='tx_cost(USD not annualized, fwd)')

# by coin

### carry

In [ ]:
carry.pivot_table(index='end_time', columns='name').xs('amtUSD', axis=1).cumsum().iplot(secondary_y=['USDT', 'total'], title=f'cumulated carry')

### weights

In [ ]:
history = copy.deepcopy(original_history)
spot_tickers = {col.split('_')[0] for col in history.columns if f"{col.split('_')[0]}_rate_funding" in history.columns}
print(spot_tickers)
leverage = 4
for spot_ticker in spot_tickers:
    coin = spot_ticker.split('USD')[0]
    stablecoin = spot_ticker.split(coin)[1]
    history[f'{spot_ticker}_carry_funding'] = history[f'{spot_ticker}_rate_funding']
    history[f'{spot_ticker}_carry_{leverage}xlong'] = leverage * history[f'{spot_ticker}_rate_funding']-(leverage-1)*history[f'{stablecoin}_rate_borrow']
    history[f'{spot_ticker}_carry_{leverage}xshort'] = leverage*(-history[f'{spot_ticker}_rate_funding']-history[f'{coin}_rate_borrow'])

carry_history = history.filter(like='_carry_funding')
USDborrow_history = history.filter(like='USDT_rate_borrow')
weight_history = weights.pivot_table(index='end_time',columns='name',values='amtUSD').drop(columns='total')

In [ ]:
joined = pd.concat([weight_history, carry_history, USDborrow_history], axis=1).resample('1d').mean()
joined.iplot(secondary_y=list(spot_tickers.intersection(joined.columns))+['USDT'])

## leverage summary

In [ ]:
avg_weights = weights.pivot_table(index='end_time', columns='name').mean().xs('amtUSD').drop(['total', 'USDT'])
print(f'leverage = {avg_weights.sum()}')
funding = original_trajectory.pivot_table(index='time', columns='name', values='funding')
averages = pd.DataFrame({'avg_weight_pct': avg_weights*100,
            'avg_funding_pct': funding.mean()*100}).sort_values('avg_weight_pct', key=abs, ascending=False)
plt.scatter(x=averages['avg_funding_pct'], y=averages['avg_weight_pct'])
averages.head(20)

## tracking error

In [ ]:
df = copy.deepcopy(original_trajectory)
df = df.set_index('time')
df['tracking_error'] = (df['RealizedCarry']-df['ExpectedCarry'])/df['previousWeight']
tracking_error = df.pivot_table(index='time', columns='name', values='tracking_error')
tracking_error.iplot(title=f'tracking error')
tracking_error.mean()

# benchmarks

In [ ]:
df = copy.deepcopy(original_trajectory)
df = df[df['name']!='USDT']
df['RealizedCarry'] /= equity
df['carry'] = df.apply(lambda x: max(-x['funding']-x['borrow'],x['funding']), axis=1)
pivot = df.pivot_table(index='time', columns='name')
benchmarks = pd.concat([pivot.xs(['carry'], axis=1), pivot[[('RealizedCarry', 'total')]]], axis=1).droplevel(0, axis=1)
stats = pd.DataFrame()
stats['vs mean'] = (benchmarks['total']-benchmarks.drop(columns='total').mean(axis=1))
stats['vs median'] = (benchmarks['total']-benchmarks.drop(columns='total').median(axis=1))
stats['vs 1sigma'] = stats['vs mean']-benchmarks.drop(columns='total').std(axis=1)
stats['vs BTC'] = (benchmarks['total']-benchmarks['BTCUSDT'])
# stats['vs ETH'] = (benchmarks['total']-benchmarks['ETHUSDT'])
# stats['vs ARB'] = (benchmarks['total']-benchmarks['ARBUSDT'])
stats.expanding().mean().iplot(title=f'perf vs benchmarks')